In [1]:
import argparse
from glob import glob
from os import makedirs, path
import pandas as pd
import pickle
import sys

from nistats.second_level_model import SecondLevelModel
from nistats.thresholding import map_threshold
from nilearn import plotting
from utils.firstlevel_utils import (get_first_level_objs, 
                                    get_first_level_maps, 
                                    load_first_level_objs, 
                                    FirstLevel)
from utils.secondlevel_utils import create_group_mask, randomise
from utils.utils import get_contrasts, get_flags

/usr/local/miniconda/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
/usr/local/miniconda/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/local/miniconda/lib/python3.7/site-packages/nipype/utils/misc.py:14: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterator
/usr/local/miniconda/lib/python3.7/site-packages/nipype/utils/

200728-22:01:54,602 duecredit ERROR:
	 Failed to import duecredit due to No module named 'duecredit'


In [2]:
# from glob import glob
# from nilearn import image
# from nipype.caching import Memory
from nipype.interfaces import fsl
# import os 
# from os import path, remove
# import shutil
# from utils.utils import get_flags

### Parse Arguments
These are not needed for the jupyter notebook, but are used after conversion to a script for production

- conversion command:
  - jupyter nbconvert --to script --execute 2ndlevel_analysis.ipynb

In [3]:
parser = argparse.ArgumentParser(description='2nd level Entrypoint Script.')
parser.add_argument('-derivatives_dir', default=None)
parser.add_argument('--tasks', nargs="+", help="Choose from ANT, CCTHot, discountFix, \
                                    DPX, motorSelectiveStop, stopSignal, \
                                    stroop, surveyMedley, twoByTwo, WATT3")
parser.add_argument('--rerun', action='store_true')
parser.add_argument('--rt', action='store_true')
parser.add_argument('--beta', action='store_true')
parser.add_argument('--n_perms', default=1000, type=int)
parser.add_argument('--quiet', '-q', action='store_true')

if '-derivatives_dir' in sys.argv or '-h' in sys.argv:
    args = parser.parse_args()
else:
    args = parser.parse_args([])
    args.derivatives_dir = '/data/derivatives/'
    args.tasks = ['stroop']
    args.rt=True
    args.n_perms = 10

In [4]:
if not args.quiet:
    def verboseprint(*args, **kwargs):
        print(*args, **kwargs)
else:
    verboseprint = lambda *a, **k: None # do-nothing function

### Setup

Organize paths and set parameters based on arguments

In [21]:
first_level_dir

'/data/derivatives/1stlevel'

In [23]:
ls /data/derivatives/1stlevel

figs/  s445/  s519/  s554/  s573/  s587/  s597/  s610/  s621/  s634/  s644/
s061/  s465/  s524/  s555/  s574/  s588/  s598/  s611/  s622/  s635/  s645/
s130/  s471/  s525/  s556/  s577/  s589/  s601/  s612/  s623/  s636/  s646/
s144/  s483/  s526/  s557/  s579/  s590/  s602/  s613/  s624/  s637/  s647/
s172/  s491/  s533/  s558/  s581/  s591/  s603/  s614/  s626/  s638/  s648/
s192/  s495/  s541/  s561/  s582/  s592/  s605/  s615/  s627/  s639/  s649/
s234/  s497/  s546/  s567/  s583/  s593/  s606/  s616/  s628/  s640/  s650/
s251/  s499/  s548/  s568/  s584/  s594/  s607/  s617/  s629/  s641/
s358/  s512/  s549/  s570/  s585/  s595/  s608/  s618/  s631/  s642/
s373/  s518/  s553/  s572/  s586/  s596/  s609/  s619/  s633/  s643/


In [5]:
# set paths
first_level_dir = path.join(args.derivatives_dir, '1stlevel')
second_level_dir = path.join(args.derivatives_dir,'2ndlevel')
fmriprep_dir = path.join(args.derivatives_dir, 'fmriprep')

# set tasks
if args.tasks is not None:
    tasks = args.tasks
else:
    tasks = ['ANT', 'CCTHot', 'discountFix',
            'DPX', 'motorSelectiveStop',
            'stopSignal', 'stroop',
            'twoByTwo', 'WATT3']
    
# set other variables
regress_rt = args.rt
beta_series = args.beta
n_perms = args.n_perms

### Create Mask

In [6]:
mask_threshold = .95
mask_loc = path.join(second_level_dir, 'group_mask_thresh-%s.nii.gz' % str(mask_threshold))
if path.exists(mask_loc) == False or args.rerun:
    verboseprint('Making group mask')
    group_mask = create_group_mask(fmriprep_dir, mask_threshold)
    makedirs(path.dirname(mask_loc), exist_ok=True)
    group_mask.to_filename(mask_loc)

### Create second level objects
Gather first level models and create second level model

In [17]:
aim1_2ndlevel_confounds_path = '../aim1_2ndlevel_regressors/aim1_2ndlevel_confounds_matrix.csv'
full_confounds_df = pd.read_csv(aim1_2ndlevel_confounds_path, index_col='index')

In [18]:
full_confounds_df

,age,sex,ANT_meanFD,CCTHot_meanFD,DPX_meanFD,WATT3_meanFD,discountFix_meanFD,motorSelectiveStop_meanFD,rest_meanFD,stopSignal_meanFD,stroop_meanFD,surveyMedley_meanFD,twoByTwo_meanFD
index,,,,,,,,,,,,,
s061,41.0,1.0,NaN,0.137224,0.108978,0.151611,0.134908,NaN,0.119460,0.125783,0.104488,0.138526,0.124104
s130,27.0,0.0,0.041484,0.031070,0.044276,0.045655,0.036948,0.036940,0.039885,0.031040,0.038328,0.030956,0.038415
s144,19.0,0.0,0.066086,0.074673,NaN,0.077678,NaN,NaN,0.057375,0.056909,NaN,NaN,0.062426
s172,19.0,0.0,0.050340,0.055910,0.068629,0.061671,0.065435,0.067503,0.050112,0.056833,0.063870,0.068802,0.054902
s192,18.0,0.0,0.094071,0.097725,0.066249,0.092005,0.069987,0.072108,0.086492,0.088668,0.067839,0.070347,0.100996
...,...,...,...,...,...,...,...,...,...,...,...,...,...
s646,22.0,1.0,0.061674,0.068604,NaN,0.065120,NaN,NaN,0.060531,0.070464,NaN,NaN,0.074248
s647,24.0,0.0,0.100468,0.090287,0.068654,0.081480,NaN,0.068188,0.061891,0.087776,0.060760,0.074409,0.090747
s648,30.0,0.0,0.061739,0.063606,0.063945,0.063606,0.074744,0.060855,0.059342,0.054841,0.053409,0.044231,0.061739


In [19]:
task_2ndlevel_df = full_confounds_df[['age', 'sex', task+'_meanFD']]
task_2ndlevel_df

,age,sex,stroop_meanFD
index,,,
s061,41.0,1.0,0.104488
s130,27.0,0.0,0.038328
s144,19.0,0.0,NaN
s172,19.0,0.0,0.063870
s192,18.0,0.0,0.067839
...,...,...,...
s646,22.0,1.0,NaN
s647,24.0,0.0,0.060760
s648,30.0,0.0,0.053409


In [30]:
ls /data/derivatives/1stlevel/s061/stroop/maps_RT-True_beta-False

contrast-RT.nii.gz  contrast-task.nii.gz


In [25]:
glob(path.join(first_level_dir, '*', task, 'maps_%s_%s/contrast-%s.nii.gz' % (rt_flag, beta_flag, contrast)))

'/data/derivatives/1stlevel/*/stroop/maps_RT-True_beta-False/contrast-task.nii.gz'

In [20]:
rt_flag, beta_flag = get_flags(regress_rt, beta_series)
for task in tasks:
    verboseprint('Running 2nd level for %s' % task)
    # load first level models
    # create contrast maps
    verboseprint('*** Creating maps')
    task_contrasts = get_contrasts(task, regress_rt)
    maps_dir = path.join(second_level_dir, task, 'secondlevel-%s_%s_maps' % (rt_flag, beta_flag))
    makedirs(maps_dir, exist_ok=True)
    # run through each contrast
    for name, contrast in task_contrasts:
        second_level_model = SecondLevelModel(mask=mask_loc, smoothing_fwhm=6)
        maps = get_first_level_maps('*', task, first_level_dir, name, regress_rt, beta_series)
        N = str(len(maps)).zfill(2)
        verboseprint('****** %s, %s files found' % (name, N))
        if len(maps) <= 1:
            verboseprint('****** No Maps')
            continue
        design_matrix = pd.DataFrame([1] * len(maps), columns=['intercept'])
        second_level_model.fit(maps, design_matrix=design_matrix)
        contrast_map = second_level_model.compute_contrast()
#         # save
#         contrast_file = path.join(maps_dir, 'contrast-%s.nii.gz' % name)
#         contrast_map.to_filename(contrast_file)
#          # write metadata
#         with open(path.join(maps_dir, 'metadata.txt'), 'a') as f:
#             f.write('Contrast-%s: %s maps\n' % (contrast, N))
#         # save corrected map
#         if n_perms > 0:
#             verboseprint('*** Running Randomise')
#             randomise(maps, maps_dir, mask_loc, n_perms=n_perms)
#             # write metadata
#             with open(path.join(maps_dir, 'metadata.txt'), 'a') as f:
#                 f.write('Contrast-%s: Randomise run with %s permutations\n' % (contrast, str(n_perms)))
#     verboseprint('Done with %s' % task)

Running 2nd level for stroop
*** Creating maps
****** congruency, 00 files found
****** No Maps


KeyboardInterrupt: 

In [ ]:
"""
# Using nistats method of first level objects. Not conducive for randomise.
rt_flag, beta_flag = get_flags(regress_rt, beta_series)
for task in tasks:
    verboseprint('Running 2nd level for %s' % task)
    # load first level models
    first_levels = load_first_level_objs(task, first_level_dir, regress_rt=regress_rt)
    if len(first_levels) == 0:
        continue
    first_level_models = [subj.fit_model for subj in first_levels]
    N = str(len(first_level_models)).zfill(2)

    # simple design for one sample test
    design_matrix = pd.DataFrame([1] * len(first_level_models), columns=['intercept'])
    
    # run second level
    verboseprint('*** Running model. %s first level files found' % N)
    second_level_model = SecondLevelModel(mask=mask_loc, smoothing_fwhm=6).fit(
        first_level_models, design_matrix=design_matrix)
    makedirs(path.join(second_level_dir, task), exist_ok=True)
    f = open(path.join(second_level_dir, task, 'secondlevel_%s_%s.pkl' % (rt_flag, beta_flag)), 'wb')
    pickle.dump(second_level_model, f)
    f.close()
    
    # create contrast maps
    verboseprint('*** Creating maps')
    task_contrasts = get_contrasts(task, regress_rt)
    maps_dir = path.join(second_level_dir, task, 'secondlevel_%s_%s_N-%s_maps' % (rt_flag, beta_flag, N))
    makedirs(maps_dir, exist_ok=True)
    for name, contrast in task_contrasts:
        verboseprint('****** %s' % name)
        contrast_map = second_level_model.compute_contrast(first_level_contrast=contrast)
        contrast_file = path.join(maps_dir, 'contrast-%s.nii.gz' % name)
        contrast_map.to_filename(contrast_file)
"""